In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [0]:
n_hidden = 35
lr = 0.01
epochs = 1000

In [0]:
string = 'hello pytorch. how long can a rnn cell remember? show me your limit!'
chars = 'abcdefghijklmnopqrstuvwxyz ?!.,:;01'

# 문자를 리스트로 바꾸고 이의 길이를 저장.
char_list = [i for i in chars]
n_letters = len(char_list)

In [0]:
# string on-hot
def string_to_onehot(strong):
  start = np.zeros(shape = n_letters, dtype = int)
  end = np.zeros(shape = n_letters, dtype = int)
  start[-2] = 1
  end[-1] = 1

  # 문자열을 차례대로 받아서 진행
  for i in string:
    # 들어온 문자가 몇 번째 문자인지 확인
    idx = char_list.index(i)
    # 0으로 배열 생성
    zero = np.zeros(shape = n_letters, dtype = int)

    # 해당 문자만 1로
    zero[idx] = 1
    # start와 새로 생긴 문자를 붙이고 start에 할당
    # 반복하면 h의 one hot, e의 one hot.. 점점 쌓이게 됨
    start = np.vstack([start, zero])
  output = np.vstack([start, end])
  return output

In [0]:
def onehot_to_word(onehot_1):
  # 텐서를 입력받고 넘파이로 바꿔줌
  onehot = torch.Tensor.numpy(onehot_1)
  # onehot 벡터 최대값 위치로 인덱스를 찾음
  return char_list[onehot.argmax()]

In [0]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.act_fn = nn.Tanh()
    
    def forward(self, input, hidden):
        # 입력과 hidden state를 cat함수로 붙여줍니다.
        combined = torch.cat((input, hidden), 1)
        # 붙인 값을 i2h 및 i2o에 통과시켜 hidden state는 업데이트, 결과값은 계산해줍니다.
        hidden = self.act_fn(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden
    
    # 아직 입력이 없을때(t=0)의 hidden state를 초기화해줍니다. 
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
rnn = RNN(n_letters, n_hidden, n_letters)

In [0]:
# 손실함수와 최적화함수를 설정

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

In [15]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
  optimizer.zero_grad()
  # 학습에 앞서 hidden state 초기화
  hidden = rnn.init_hidden()

  # 문자열 전체에 대한 손실을 구하기 위해 total_loss 만들어줌
  total_loss = 0
  for j in range(one_hot.size()[0] - 1):
    # 입력은 앞의 글자
    # p y t o r c
    input_ = one_hot[j : j + 1, :]
    # 목표값은 뒤의 글자
    # y t o r c h
    target = one_hot[j + 1]
    output, hidden = rnn.forward(input_, hidden)

    # 다 1차원으로 만듦
    loss = loss_func(output.view(-1), target.view(-1))
    total_loss += loss

  total_loss.backward()
  optimizer.step()

  if i % 10 == 0:
    print(total_loss)

tensor(2.8486, grad_fn=<AddBackward0>)
tensor(1.2048, grad_fn=<AddBackward0>)
tensor(0.6914, grad_fn=<AddBackward0>)
tensor(0.4150, grad_fn=<AddBackward0>)
tensor(0.2740, grad_fn=<AddBackward0>)
tensor(0.2012, grad_fn=<AddBackward0>)
tensor(0.1492, grad_fn=<AddBackward0>)
tensor(0.1236, grad_fn=<AddBackward0>)
tensor(0.0960, grad_fn=<AddBackward0>)
tensor(0.0857, grad_fn=<AddBackward0>)
tensor(0.0675, grad_fn=<AddBackward0>)
tensor(0.0621, grad_fn=<AddBackward0>)
tensor(0.0506, grad_fn=<AddBackward0>)
tensor(0.0452, grad_fn=<AddBackward0>)
tensor(0.0401, grad_fn=<AddBackward0>)
tensor(0.0425, grad_fn=<AddBackward0>)
tensor(0.0342, grad_fn=<AddBackward0>)
tensor(0.0306, grad_fn=<AddBackward0>)
tensor(0.0448, grad_fn=<AddBackward0>)
tensor(0.0297, grad_fn=<AddBackward0>)
tensor(0.0271, grad_fn=<AddBackward0>)
tensor(0.0244, grad_fn=<AddBackward0>)
tensor(0.0227, grad_fn=<AddBackward0>)
tensor(0.0238, grad_fn=<AddBackward0>)
tensor(0.0211, grad_fn=<AddBackward0>)
tensor(0.0220, grad_fn=<A

In [16]:
start = torch.zeros(1, n_letters)
start[:, -2] = 1

with torch.no_grad():
  hidden = rnn.init_hidden()
  # 입력으로 start 토큰 전달
  input_ = start
  # output string에 문자열 계속 붙여줌
  output_string = ''

  for i in range(len(string)):
    output, hidden = rnn.forward(input_, hidden)
    # 결과값을 문자로 바꿔서 output_string에 붙여줌
    output_string += onehot_to_word(output.data)
    # 이번 결괏값이 다음의 입력값
    input_ = output

print(output_string)

hello pytorch. moyoooongnnpyongonggnngombrrmloe cnyonaoaalrmbrmelnnl
